# Tensoflow NLP Model for Text Classification

## Imports

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split
import pickle

from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import os
import datetime
# import tensorflow_hub as hub
from tensorflow import keras

## Pre-Processing

In [2]:
with open('../data/processed_20_data.pkl', 'rb') as file:
    processed_20_data = pickle.load(file)

In [3]:
#only keep relevant columns
processed_20_data = processed_20_data[['detailed_type', 'public_description']]

In [4]:
processed_20_data = processed_20_data[(processed_20_data['detailed_type'] != 'TRAFFIC MAINTENANCE - OTHER') & (processed_20_data['detailed_type'] != 'Red Zone') & (processed_20_data['detailed_type'] != 'Oversized Vehicle') & ((processed_22_data['detailed_type'] != 'Container Request'))]

NameError: name 'processed_22_data' is not defined

In [ ]:
# processed_21_data[processed_21_data['public_description'] == 'Encampment out front of Kristys.']

In [ ]:
# #split train and test data
# (requests_train, requests_test) = train_test_split(processed_21_data, test_size = 0.2, random_state = 42)

## Tensorflow Model

### Imports

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import os
import datetime
import tensorflow_hub as hub

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
processed_20_data.sample(10)

In [ ]:
X_train, X_test = train_test_split(processed_20_data, test_size=0.2, random_state=42)

In [ ]:
from sklearn.utils import class_weight

In [ ]:
class_weights = list(class_weight.compute_class_weight(class_weight = 'balanced',
                                                       classes= np.unique(processed_20_data['detailed_type']),
                                                      y=processed_20_data['detailed_type']
                                                      ))

In [ ]:
len(X_train['detailed_type'].value_counts())

In [ ]:
class_weights

In [ ]:
num_options = len(class_weights)
num_options

In [ ]:
weights = {}

In [ ]:
for index, weight in enumerate(class_weights) :
    weights[index]=weight

In [ ]:
weights

In [ ]:
list1 = [3,1]

In [ ]:
np.array(list1).shape

In [ ]:
array1 = np.array([1])
array2 = np.array([5])

In [ ]:
# string1 = 'string1'
# string2 = 'string2'
# string3 = 'string3'

# string_array = np.array([string1])
# print('shape', string_array.shape)
# print('type', type(string_array))

In [ ]:
# test_array = X_train['public_description'].values[0:1]
# print('shape', test_array.shape)
# print('type', type(test_array))

In [ ]:
# type(X_train['public_description'].values[0])

In [ ]:
dataset_train = tf.data.Dataset.from_tensor_slices((string_array, test_array))

In [ ]:
#Instantiate
dataset_train = tf.data.Dataset.from_tensor_slices((X_train['public_description'].values, X_train['detailed_type'].values))
dataset_test = tf.data.Dataset.from_tensor_slices((X_test['public_description'].values, X_test['detailed_type'].values))

In [ ]:
# dataset_train = tf.data.Dataset.from_tensor_slices((testing, X_train['detailed_type'].values))

In [ ]:
for text, target in dataset_train.take(5):
    print('description: {}, Target: {}'.format(text, target))

In [ ]:
for text, target in dataset_test.take(5):
    print('description: {}, Target: {}'.format(text, target))

In [ ]:
detailed_types = list(np.unique(X_train['detailed_type']))
detailed_types

In [ ]:
vals = list(range(num_options))

In [ ]:
num_options == len(vals)

In [ ]:
table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant(detailed_types),
        values=tf.constant(vals),
    ),
        default_value=tf.constant(-1),
        name="target_encoding"
)


In [ ]:
pickle.dump(detailed_types, open('../data/detailed_types.pkl', 'wb'))
pickle.dump(vals, open('../data/vals.pkl', 'wb'))

In [ ]:
@tf.function
def target(x):
    return table.lookup(x)

In [ ]:
def show_batch(dataset, size=5):
    for batch, label in dataset.take(size):
        print(batch.numpy())
        print(target(label).numpy())

In [ ]:
show_batch(dataset_test, 6)

In [ ]:
def fetch(text, labels):
        return text, tf.one_hot(target(labels),num_options)

In [ ]:
train_data_f = dataset_train.map(fetch)
test_data_f=dataset_test.map(fetch)

In [ ]:
# next(iter(train_data_f))

In [ ]:
train_data, train_labels = next(iter(train_data_f.batch(2)))
train_data, train_labels

In [ ]:
# embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"
# hub_layer = hub.KerasLayer(embedding, output_shape=[128], input_shape=[],
#                            dtype=tf.string, trainable=True)
# hub_layer(train_data[:1])

In [ ]:
# model = tf.keras.Sequential()
# model.add(hub_layer)
# for units in [128, 128, 64, 32]:
#     model.add(tf.keras.layers.Dense(units, activation='relu'))
#     model.add(tf.keras.layers.Dropout(0.3))
# model.add(tf.keras.layers.Dense(num_options, activation='softmax'))

# model.summary()

In [ ]:
# model.compile(optimizer='adam',
#              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
#              metrics=['accuracy'])

In [ ]:
train_data_f=train_data_f.shuffle(70000).batch(512)
test_data_f=test_data_f.batch(512)

In [ ]:
model = keras.models.load_model('../data/tf_model_22.pkl')

In [ ]:
# history = model.train(train_data_f,
#                    epochs=5,
#                    validation_data=test_data_f,
#                    verbose=1,
#                    class_weight=weights)

In [ ]:
# update neural network with new data only
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

# define dataset
# X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# record the number of input features in the data
# n_features = X.shape[1]
# split into old and new data
# X_old, X_new, y_old, y_new = train_test_split(X, y, test_size=0.50, random_state=1)
# define the model
# model = Sequential()
# model.add(Dense(20, kernel_initializer='he_normal', activation='relu', input_dim=n_features))
# model.add(Dense(10, kernel_initializer='he_normal', activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# # define the optimization algorithm
# opt = SGD(learning_rate=0.01, momentum=0.9)
# # compile the model
# model.compile(optimizer=opt, loss='binary_crossentropy')
# # fit the model on old data
# model.fit(X_old, y_old, epochs=150, batch_size=32, verbose=0)
 
# save model...
 
# load model...
 
# update model on new data only with a smaller learning rate
opt = SGD(learning_rate=0.001, momentum=0.9)
# compile the model
model.compile(optimizer=opt, loss='binary_crossentropy')
# fit the model on new data
model.fit(train_data, train_labels, epochs=3, batch_size=32, verbose=0)

In [ ]:
# model.save('../data/tf_model.pkl')

In [ ]:
# pickle.dump(model, open('../data/tf_model.pkl', 'wb'))

In [ ]:
for_results = len(list(dataset_test))

In [ ]:
results = model.evaluate(dataset_test.map(fetch).batch(for_results), verbose=2)

In [ ]:
print(results)

In [ ]:
next(iter(dataset_test.map(fetch).batch(1)))

In [ ]:
type(dataset_test)

In [ ]:
test_data, test_labels = next(iter(dataset_test.map(fetch).batch(for_results)))

In [ ]:
y_pred=model.predict(test_data)

In [ ]:
# type(test_data)

In [ ]:
y_pred

In [ ]:
results_df = pd.DataFrame(y_pred)
results_df.columns = ['72 Hour Violation',
 'COVID-19',
 'Container Left Out',
 'Container Out Late',
 'Damaged/Knocked Over Pole',
 'Dead Animal',
 'Development Services - Code Enforcement',
 'Drain Inlet',
 'Encampment',
 'Encroachment',
 'Environmental Services Code Compliance',
 'Fallen/Hanging Tree Limb',
 'Flashing Traffic Signal Lights',
 'Graffiti Removal',
 'Graffiti Removal - Commercial',
 'Illegal Dumping',
 'Litter',
 'Missed Collection',
 'Other',
 'Oversized Vehicle',
 'Parking',
 'Parking Zone Violation',
 'Pavement Maintenance',
 'Potential Missed Collection',
 'Pothole',
 'Quality of Life Issues',
 'ROW Maintenance',
 'Resurfacing Evaluation',
 'Right-of-Way Code Enforcement',
 'Shared Mobility Device',
 'Sidewalk Repair Issue',
 'Stormwater',
 'Stormwater Code Enforcement',
 'Stormwater Pollution Prevention',
 'Street Flooded',
 'Street Light Maintenance',
 'Street Light Out of Service',
 'Street Sweeping',
 'Traffic Engineering',
 'Traffic Sign Maintenance',
 'Traffic Signal Issue',
 'Traffic Signal Out of Service',
 'Traffic Signal Timing',
 'Trash/Recycling Collection',
 'Tree Maintenance',
 'Tree Removal',
 'Tree Trimming for Pedestrian/Vehicle Clearance',
 'Trimming Request',
 'Vegetation Encroachment',
 'Waste on Private Property',
 'Weed Cleanup']
results_df

In [ ]:
#return column where value is that above
j = 0
max_col = []

for j in range(100):

    i=0


    while i < results_df.shape[1]:
        if results_df.iloc[j,i] == results_df.iloc[j,:].max():
            max_col.append(list(results_df.columns)[i])
        i += 1

In [ ]:
max_col

In [ ]:
test_df = pd.DataFrame(test_data).head(100)

In [ ]:
test_df['prediction'] = max_col

In [ ]:
test_df.sample(10)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(test_labels.numpy().argmax(axis=1), y_pred.argmax(axis=1), zero_division = 1))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_labels.numpy().argmax(axis=1), y_pred.argmax(axis=1))

In [ ]:
model.save('../data/tf_model_20.pkl')